In [50]:
# Colab setup ------------------
import os, sys, subprocess
# ------------------------------

import numpy as np
import scipy.integrate

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting

# Set to True to have fully interactive plots with Python;
# Set to False to use pre-built JavaScript-based plots
interactive_python_plots = False
notebook_url = "localhost:8888"

bokeh.io.output_notebook()

def callback(attr, old, new):
    for i in range(len(gata6)):
        ss = scipy.integrate.odeint(rhs, [foxa2_initial, foxf1_initial], t, args=(gata6[i],
                                                                            foxa2_vmax_slider.value,
                                                                            foxf1_vmax_slider.value,
                                                                            gata6_a2_km,
                                                                            gata6_a2_n,
                                                                            foxf1_a2_km_slider.value,
                                                                            foxf1_a2_n_slider.value,
                                                                            foxa2_a2_km_slider.value,
                                                                            foxa2_a2_n_slider.value,
                                                                            gata6_a2_km,
                                                                            gata6_a2_n,
                                                                            foxa2_f1_km_slider.value,
                                                                            foxa2_f1_n_slider.value,
                                                                            foxf1_f1_km_slider.value,
                                                                            foxf1_f1_n_slider.value,))
        foxa2s[i] = ss[-1,0]
        foxf1s[i] = ss[-1,1]
    cds.data["foxa2"] = foxa2s
    cds.data["foxf1"] = foxf1s
    
def foxa2_hill(GATA6, FOXA2, FOXF1, foxa2_max, GATA6_Km, GATA6_n, FOXF1_Km, FOXF1_n, FOXA2_Km, FOXA2_n):
    return foxa2_max * (((GATA6/GATA6_Km) ** GATA6_n) * ((FOXA2/FOXA2_Km) ** FOXA2_n)) / ((1 + (FOXF1/FOXF1_Km) ** FOXF1_n) * (1 + (GATA6/GATA6_Km) ** GATA6_n) * (1 + (FOXA2/FOXA2_Km) ** FOXA2_n)) + 0.1 - 0.2 * FOXA2

def foxf1_hill(GATA6, FOXF1, FOXA2, foxf1_max, GATA6_Km, GATA6_n, FOXA2_Km, FOXA2_n, FOXF1_Km, FOXF1_n):
    return foxf1_max * (((GATA6/GATA6_Km) ** GATA6_n) * ((FOXF1/FOXF1_Km) ** FOXF1_n)) / ((1 + (FOXA2/FOXA2_Km) ** FOXA2_n) * (1 + (GATA6/GATA6_Km) ** GATA6_n) * (1 + (FOXF1/FOXF1_Km) ** FOXF1_n)) + 0.1 - 0.2 * FOXF1

def rhs(X, t, GATA6, foxa2_max, foxf1_max, gata6_a2_km, gata6_a2_n, foxf1_a2_km, foxf1_a2_n, foxa2_a2_km, foxa2_a2_n, 
        gata6_f1_km, gata6_f1_n, foxa2_f1_km, foxa2_f1_n, foxf1_f1_km, fox_f1_f1_n):
    FOXA2, FOXF1 = X
    foxa2 = foxa2_hill(GATA6, FOXA2, FOXF1, foxa2_max, gata6_a2_km, gata6_a2_n, foxf1_a2_km, foxf1_a2_n, foxa2_a2_km, foxa2_a2_n)
    foxf1 = foxf1_hill(GATA6, FOXF1, FOXA2, foxf1_max, gata6_f1_km, gata6_f1_n, foxa2_f1_km, foxa2_f1_n, foxf1_f1_km, fox_f1_f1_n)
    return [foxa2, foxf1]

foxa2_vmax_slider = bokeh.models.Slider(
    title="foxa2 vmax", start=0, end=50, step=0.01, value=12.0, width=150
)
foxf1_vmax_slider = bokeh.models.Slider(
    title="foxf1 vmax", start=0, end=50, step=0.01, value=12.3, width=150
)
foxf1_a2_km_slider = bokeh.models.Slider(
    title="foxf1->a2 Km", start=0.01, end=50, step=0.01, value=0.46, width=150
)
foxf1_a2_n_slider = bokeh.models.Slider(
    title="foxf1->a2 n", start=0, end=50, step=0.1, value=0.20, width=150
)
foxa2_a2_km_slider = bokeh.models.Slider(
    title="foxa2->a2 Km", start=0.01, end=50, step=0.01, value=4.42, width=150
)
foxa2_a2_n_slider = bokeh.models.Slider(
    title="foxa2->a2 n", start=0, end=50, step=0.1, value=1.2, width=150
)
foxf1_f1_km_slider = bokeh.models.Slider(
    title="foxf1->f1 Km", start=0.01, end=50, step=0.01, value=5.41, width=150
)
foxf1_f1_n_slider = bokeh.models.Slider(
    title="foxf1->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)
foxa2_f1_km_slider = bokeh.models.Slider(
    title="foxa2->f1 Km", start=0.01, end=50, step=0.01, value=6.67, width=150
)
foxa2_f1_n_slider = bokeh.models.Slider(
    title="foxa2->f1 n", start=0, end=50, step=0.1, value=1.0, width=150
)


# Set up time points
gata6_max = 40
gata6 = np.linspace(0, gata6_max, gata6_max+1)
t = np.linspace(0, 720, 720)

# GATA6 specific parameters
gata6_a2_km = 10
gata6_f1_km = 4.5

gata6_a2_n = 2
gata6_f1_n = 1

foxa2_initial = 0
foxf1_initial = 0

foxa2s = np.zeros(len(gata6))
foxf1s = np.zeros(len(gata6))

# Build s, taking slider values are parameters.

for i in range(len(gata6)):
    ss = scipy.integrate.odeint(rhs, [foxa2_initial, foxf1_initial], t, args=(gata6[i],
                                                                        foxa2_vmax_slider.value,
                                                                        foxf1_vmax_slider.value,
                                                                        gata6_a2_km,
                                                                        gata6_a2_n,
                                                                        foxf1_a2_km_slider.value,
                                                                        foxf1_a2_n_slider.value,
                                                                        foxa2_a2_km_slider.value,
                                                                        foxa2_a2_n_slider.value,
                                                                        gata6_a2_km,
                                                                        gata6_a2_n,
                                                                        foxa2_f1_km_slider.value,
                                                                        foxa2_f1_n_slider.value,
                                                                        foxf1_f1_km_slider.value,
                                                                        foxf1_f1_n_slider.value,))
    foxa2s[i] = ss[-1,0]
    foxf1s[i] = ss[-1,1]

# Place the data in a ColumnDataSource
cds = bokeh.models.ColumnDataSource(dict(gata6=gata6, foxa2=foxa2s, foxf1=foxf1s))

# Build the plot
p = bokeh.plotting.figure(
    frame_height=400,
    frame_width=400,
    x_axis_label="GATA6",
    y_axis_label="GATA6 Response",
    x_range=[0, 30],
    y_range=[-0.02, 20],
)
p.line(source=cds, x="gata6", y="foxa2", line_width=2, color='red', legend_label="FOXA2");
p.line(source=cds, x="gata6", y="foxf1", line_width=2, color='green', legend_label="FOXF1")

foxa2_vmax_slider.on_change("value", callback)
foxf1_vmax_slider.on_change("value", callback)
foxf1_a2_km_slider.on_change("value", callback)
foxf1_a2_n_slider.on_change("value", callback)
foxa2_a2_km_slider.on_change("value", callback)
foxa2_a2_n_slider.on_change("value", callback)
foxf1_f1_km_slider.on_change("value", callback)
foxf1_f1_n_slider.on_change("value", callback)
foxa2_f1_km_slider.on_change("value", callback)
foxa2_f1_n_slider.on_change("value", callback)

layout = bokeh.layouts.row(
    p,
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(foxa2_vmax_slider, foxf1_a2_km_slider, foxf1_a2_n_slider, foxa2_a2_km_slider, foxa2_a2_n_slider),
    bokeh.models.Spacer(width=30),
    bokeh.layouts.column(foxf1_vmax_slider, foxf1_f1_km_slider, foxf1_f1_n_slider, foxa2_f1_km_slider, foxa2_f1_n_slider)
)

def app(doc):
    doc.add_root(layout)

Loading BokehJS ...

In [51]:
interactive_python_plots = True

if interactive_python_plots:
    bokeh.io.show(app, notebook_url=notebook_url)
else:
    bokeh.io.show(biocircuits.jsplots.autorepressor_response_to_pulse())

In [49]:
print(cds.data)

{'gata6': array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40.]), 'foxa2': array([ 0.5       ,  0.52204643,  0.59624434,  0.74873814,  1.03876463,
        1.66114372,  2.8454092 ,  4.32242274,  5.79021229,  7.15703073,
        8.40019842,  9.51737386, 10.51426985, 11.40018958, 12.18583   ,
       12.88206862, 13.49929612, 14.04707856, 14.53401967, 14.96774077,
       15.35492587, 15.70139937, 16.01221652, 16.29175548, 16.5438045 ,
       16.7716416 , 16.97810526, 17.16565644, 17.33643214, 17.49229153,
       17.63485535, 17.76553961, 17.8855842 , 17.99607733, 18.09797632,
       18.19212527, 18.27927024, 18.36007209, 18.43511756, 18.50492877,
       18.56997129]), 'foxf1': array([0.5       , 0.54920976, 0.74994527, 1.37461693, 2.94759126,
       4.94780554, 6.30854107, 7.07586392, 7.59369769, 7.9820457 ,
     